In [1]:
import requests
import collections
import re
from datetime import date
import datetime,time
from dateutil.rrule import rrule, DAILY, HOURLY
import pandas as pd
from utils import *
import sys

#sys.path.append('/usr/local/lib/python2.7/site-packages/')

In [2]:
import logging
logger=logging.getLogger()
logging.basicConfig(filename='./log.log',level=logging.WARNING)
#logger.setLevel(logging.WARNING)

Check quota [here](https://console.developers.google.com/project/childmarriage-1019/apiui/apiview/youtube/quotas)

In [18]:
taxonomyWords=['child marriage','child bride','underage bride','teen bride','forced marriage','early marriage']
taxonomyWords.extend([u'बाल विवाह',u'बालिका वधु',u'नाबालिक विवाह',u'नाबालिक वधु',u'किशोर वधु',u'किशोर बीवी',u'जबरन शादी',u'जबरन विवाह'])
taxonomyWords=map(lambda x:'"'+x+'"',taxonomyWords)
FULLQUERY='|'.join(taxonomyWords)

In [19]:
FULLQUERY

u'"child marriage"|"child bride"|"underage bride"|"teen bride"|"forced marriage"|"early marriage"|"\u092c\u093e\u0932 \u0935\u093f\u0935\u093e\u0939"|"\u092c\u093e\u0932\u093f\u0915\u093e \u0935\u0927\u0941"|"\u0928\u093e\u092c\u093e\u0932\u093f\u0915 \u0935\u093f\u0935\u093e\u0939"|"\u0928\u093e\u092c\u093e\u0932\u093f\u0915 \u0935\u0927\u0941"|"\u0915\u093f\u0936\u094b\u0930 \u0935\u0927\u0941"|"\u0915\u093f\u0936\u094b\u0930 \u092c\u0940\u0935\u0940"|"\u091c\u092c\u0930\u0928 \u0936\u093e\u0926\u0940"|"\u091c\u092c\u0930\u0928 \u0935\u093f\u0935\u093e\u0939"'

In [20]:
QUERY=re.escape('"child marriage"|"child bride"')
from secrets import KEY

diff=1

In [21]:
def getVideoParts(vData):
    title=res['items'][0]['snippet'].get('title')
    time=res['items'][0]['snippet'].get('publishedAt')
    tags=res['items'][0]['snippet'].get('tags')
    lang=res['items'][0]['snippet'].get('defaultAudioLanguage')
    description=res['items'][0]['snippet'].get('description')
    commentCount=viewCount=favouriteCount=dislikeCount=likeCount=None
    
    stats=res['items'][0].get('statistics')

    commentCount=stats.get('commentCount')
    viewCount=stats.get('viewCount')
    favouriteCount=stats.get('favoriteCount')
    dislikeCount=stats.get('dislikeCount')
    likeCount=stats.get('likeCount')
    
    return time,title,tags,lang,description,commentCount,viewCount,favouriteCount,dislikeCount,likeCount

In [22]:
def getCommentsFromVideo(vid):
    requestString='https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId='+vid+'&key='+KEY

    res=requests.get(requestString)
    commentData=res.json()

    if res.status_code==403:
        # Comments disabled
        logging.warning('Comments error %d,%s' % (res.status_code,vid))
        logging.warning(commentData)
        return [],None
    
    isMore=False
    
    nextToken=commentData.get('nextPageToken')
    
    if nextToken:
        if len(nextToken)>0:
            logging.warning('!!!!More than 100 comments....')
            isMore=True
    
    try:
        logging.info('Got %d comments' % (len(commentData['items'])))
    except:
        logging.warning('!!!!ERROR',commentData.keys())
        logging.warning('%d' % res.status_code)
        logging.warning(commentData)
    
    return commentData['items'],isMore

In [23]:
def getCountryFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('country')
    else:
        return None

In [24]:
def getDescriptionFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('description')
    else:
        return None

In [25]:
def getTitleFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('title')
    else:
        return None

In [26]:
def getVideoInfoFromVideo(vid):

        requestString='https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id='+vid+'&key='+KEY

        logging.info('QUERYING VIDEO %s' % vid)

        d=requests.get(requestString)

        return d.json()

In [27]:
def getChannelInfoFromChannel(cid):
    requestString='https://www.googleapis.com/youtube/v3/channels?part=snippet&id='+cid+'&key='+KEY

    logging.info('QUERYING CHANNEL'+cid)

    d=requests.get(requestString)

    return d.json()

In [28]:
def getRepliesFromComment(commentId):
    requestString='https://www.googleapis.com/youtube/v3/comments?part=snippet&maxResults=100&parentId='+commentId+\
    '&key='+KEY
    
    isMore=False
    
    d=requests.get(requestString)
    
    returnData=d.json()
    
    nextToken=returnData.get('nextPageToken')
    
    if nextToken:
        if len(nextToken)>0:
            logging.warning('!!!!More than 100 comments....')
            isMore=True
    
    return returnData,isMore

In [34]:

nDayDiff=7*4
# Look back n days from start date

diff=1
# Start date is yesterday

now=datetime.datetime.now()

endDate=date(now.year,now.month,now.day)
# Throw away hour, minute, second

endDate=endDate-datetime.timedelta(days=diff)
endDateTemp=endDate
# Look back diff days

startDate=endDate-datetime.timedelta(days=nDayDiff)
startDateTemp=startDate

endDate=endDate.strftime("%Y-%m-%dT%H:%M:%SZ")
startDate=startDate.strftime("%Y-%m-%dT%H:%M:%SZ")

startDates=[startDate]
endDates=[endDate]

logging.info('%s - %s' %  (' '.join(startDates),' '.join(endDates)))

startDates=[d for d in rrule(DAILY,dtstart=startDateTemp,until=endDateTemp-datetime.timedelta(days=1))]
startDates=[d.strftime("%Y-%m-%dT%H:%M:%SZ") for d in startDates]

logging.info('Start dates %s' % (' '.join(startDates)))

vIdList=[]
vTimeList=[]
vTitleList=[]
vLangList=[]
vTagList=[]
vDescriptionList=[]
vCommentCountList=[]
vViewCountList=[]
vFavouriteCountList=[]
vDislikeCountList=[]
vLikeCountList=[]

cIdList=[]
cCountryList=[]
cDescriptionList=[]
cTitleList=[]

commentIdList=[]
commentVideoIdList=[]
commentTextList=[]
commentUserIdList=[]
commentLikesList=[]
commentTimeList=[]

# commentCount:0 viewCount:4 favoriteCount:0 dislikeCount:0 likeCount:0

replyTextList=[]
replyCommentIdList=[]
replyTimeList=[]
replyAuthorIdList=[]
replyLikeCountList=[]

##############
for start,end in zip(startDates[0:-1],startDates[1:]):

    requestString='https://www.googleapis.com/youtube/v3/search?part=snippet&q='+FULLQUERY+'&key='+KEY+'&maxResults=50&type=video&publishedBefore='+end+'&publishedAfter='+start

    print 'QUERYING API IN RANGE',start,end
    logging.info('QUERYING API IN RANGE %s - %s' % (start,end))
    
    d=requests.get(requestString)
    logging.info('Search return code %d' % d.status_code)

    data=d.json()
    
    logging.info('Got %d videos' % (len(data['items'])))

    videoIds=[]

    for v,video in enumerate(data['items']):
        
        ############################################
        # Video stuff

        videoId=video[u'id']['videoId']
        vIdList.append(videoId)
        
        videoIds.append(videoId)
        videoData=getVideoData(videoId)

        logging.info('Video ID %s' % videoId)
        logging.info('Video keys %s' % (videoData['items'][0].keys()))

        res=getVideoInfoFromVideo(videoData['items'][0]['id'])
        channelId=res['items'][0]['snippet']['channelId']

        vTime,vTitle,vTags,vLang,vDescription,vCommentCount,vViewCount,vFavouriteCount,vDislikeCount,vLikeCount=\
        getVideoParts(res['items'][0])
        
        logging.info('Video Title: %s' % vTitle)
        vTitleList.append(vTitle)
        
        logging.info('Video time: %s' % vTime)
        vTimeList.append(vTime)
        
        logging.info('Video tags: %s' % vTags)
        vTagList.append(vTags)
        
        if vLang:logging.info('Video lang: %s' % vLang)
        vLangList.append(vLang)
        
        if False:print 'Video Description:',vDescription
        vDescriptionList.append(vDescription)
        
        logging.info('Video Stats %s' % ' '.join([k+':'+v for k,v in zip(['comments','views','favs','dislike','like'],[vCommentCount,vViewCount,vFavouriteCount,vDislikeCount,vLikeCount]) if v]))
        vCommentCountList.append(vCommentCount)
        vViewCountList.append(vViewCount)
        vFavouriteCountList.append(vFavouriteCount)
        vDislikeCountList.append(vDislikeCount)
        vLikeCountList.append(vLikeCount)
        
        commentThreads,isMoreComments=getCommentsFromVideo(videoId)
        
        for thread in commentThreads:
            
            commentId=thread['snippet']['topLevelComment']['id']
            commentIdList.append(commentId)
            
            commentVideoId=thread['snippet']['topLevelComment']['snippet']['videoId']
            commentVideoIdList.append(commentVideoId)
            
            commentText=thread['snippet']['topLevelComment']['snippet']['textDisplay']
            commentTextList.append(commentText)
            
            commentTime=thread['snippet']['topLevelComment']['snippet']['publishedAt']
            commentTimeList.append(commentTime)
            
            try:commentUser=thread['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
            except:commentuser=None
            
            commentUserIdList.append(commentUser)
            
            commentLikes=thread['snippet']['topLevelComment']['snippet']['likeCount']
            commentLikesList.append(commentLikes)
                    
            if thread['snippet']['totalReplyCount']>0:
                replies,isMoreReplies=getRepliesFromComment(commentId)
                logging.info('%d replies' % (len(replies)))
                
                for reply in replies['items']:
                    replyText=reply['snippet']['textDisplay']
                    replyTextList.append(replyText)
                    
                    replyCommentId=reply['snippet']['parentId']
                    replyCommentIdList.append(replyCommentId)
                    
                    replyTime=reply['snippet']['publishedAt']
                    replyTimeList.append(replyTime)
                    
                    try:replyAuthorId=reply['snippet']['authorChannelId']['value']
                    except:
                        logging.warning('Reply no author id %s' % reply['snippet'])
                        replyAuthorId=None
                    replyAuthorIdList.append(replyAuthorId)
                    
                    replyLikeCount=reply['snippet']['likeCount']
                    replyLikeCountList.append(replyLikeCount)
                
        ############################################
        # Channel stuff
        
        info=getChannelInfoFromChannel(channelId)
        cIdList.append(channelId)

        cCountry=getCountryFromInfo(info)
        logging.info('Channel Country: %s' % cCountry)
        cCountryList.append(cCountry)
        
        cDescription=getDescriptionFromInfo(info)
        if False:print 'Channel Description:',cDescription
        cDescriptionList.append(cDescription)
        
        cTitle=getTitleFromInfo(info)
        logging.info('Channel Title %s' % cTitle)
        cTitleList.append(cTitle)
        
        logging.info('\n')
        
        assert len(vIdList)==len(vTitleList)==len(vTagList)==len(vLangList)==len(vDescriptionList)==len(vViewCountList)\
        ==len(vFavouriteCountList)==len(vDislikeCountList)==len(vLikeCountList)==len(vTimeList)\
        ,'Video data mismatched'
        
        assert len(commentIdList)==len(commentLikesList)==len(commentTextList)==len(commentTimeList)\
        ==len(commentUserIdList)==len(commentVideoIdList)\
        ,'Comment data mismatched'
        
        assert len(replyAuthorIdList)==len(replyCommentIdList)==len(replyLikeCountList)==len(replyTextList)\
        ==len(replyTimeList)\
        ,'Reply data mismatched'
        
    logging.warning('Stats: %d videos %d comments %d replies' %(len(vIdList),len(commentIdList),len(replyCommentId)))

videoDf=pd.DataFrame(data={'id':vIdList,'title':vTitleList,'tags':vTagList,'lang':vLangList,'description':vDescriptionList,\
                'views':vViewCountList,'favourites':vFavouriteCountList,'dislikes':vDislikeCountList,\
                'likes':vLikeCountList,'channelId':cIdList,'channelCountry':cCountryList,\
                'channelDescription':cDescriptionList,'channelTitle':cTitleList},index=vTimeList)       
      
commentDf=pd.DataFrame(data={'text':commentTextList,'videoId':commentVideoIdList,'id':commentIdList,\
                             'likes':commentLikesList,'user':commentUserIdList},index=commentTimeList)

replyDf=pd.DataFrame(data={'text':replyTextList,'likes':replyLikeCountList,'author':replyAuthorIdList,\
                           'parent':replyCommentIdList},index=replyTimeList)

QUERYING API IN RANGE 2015-07-07T00:00:00Z 2015-07-08T00:00:00Z
QUERYING API IN RANGE

 2015-07-08T00:00:00Z 2015-07-09T00:00:00Z
QUERYING API IN RANGE

 2015-07-09T00:00:00Z 2015-07-10T00:00:00Z
QUERYING API IN RANGE

 2015-07-10T00:00:00Z 2015-07-11T00:00:00Z
QUERYING API IN RANGE

 2015-07-11T00:00:00Z 2015-07-12T00:00:00Z
QUERYING API IN RANGE

...

 2015-07-29T00:00:00Z 2015-07-30T00:00:00Z
QUERYING API IN RANGE

 2015-07-30T00:00:00Z 2015-07-31T00:00:00Z
QUERYING API IN RANGE

 2015-07-31T00:00:00Z 2015-08-01T00:00:00Z
QUERYING API IN RANGE

 2015-08-01T00:00:00Z 2015-08-02T00:00:00Z
QUERYING API IN RANGE

 2015-08-02T00:00:00Z 2015-08-03T00:00:00Z


In [30]:
print replyDf.shape,videoDf.shape,commentDf.shape

(15, 4) (300, 13) (31, 5)


###Write Out

In [35]:
videoDf.to_pickle('videos.dat')
commentDf.to_pickle('comments.dat')
replyDf.to_pickle('replies.dat')

###Read In

In [3]:
videoDf=pd.read_pickle('videos.dat')
commentDf=pd.read_pickle('comments.dat')
replyDf=pd.read_pickle('replies.dat')

In [6]:
videoDf.index=pd.to_datetime(videoDf.index)

In [8]:
videoDf['id'].resample('d',how='count')

2015-07-07    50
2015-07-08    50
2015-07-09    50
2015-07-10    50
2015-07-11    50
2015-07-12    50
2015-07-13    50
2015-07-14    50
2015-07-15    50
2015-07-16    50
2015-07-17    50
2015-07-18    50
2015-07-19    50
2015-07-20    50
2015-07-21    50
2015-07-22    50
2015-07-23    50
2015-07-24    50
2015-07-25    50
2015-07-26    50
2015-07-27    50
2015-07-28    50
2015-07-29    50
2015-07-30    50
2015-07-31    50
2015-08-01    50
2015-08-02    50
Freq: D, Name: id, dtype: int64

In [11]:
commentDf.videoId.value_counts()

TGwTeR8Y8Qw    100
MgnnQ2CN6yY     99
2y70CqPE_cM     79
Al4-THYps_w     55
xkFTZcUPjBg     52
sDVbAD5QCrk     39
hreEjmTIeKM     33
GjV10YBi1UQ     29
mFK6k7ClkIc     27
-s4S0njqrdE     22
kN_kAkmVJvs     18
l5jl-UAtEc8     17
z1dcbQGJuiA     17
qE0f0Hq0Voo     14
Yz9Txg1I6cs     12
oTtQ5tUf_6U     11
flNywB5uMt4     11
RFxgvG2CQcc      9
vnY5Ns5CxPk      8
hBVjZfBnW-4      8
nbTVFW0LTnI      8
5xIF0aOe198      8
70w98yeKj74      8
2pnfP7tW494      8
1Q88H5lYEko      7
w9gsiNjX8UA      7
qbpkO5lBOWM      6
E3DKqzGQQz8      6
Lh063DA601c      5
jJQdexuPpAM      4
              ... 
nl-NfmTczFc      1
ZSI9aH6aMbE      1
uinbPS2mkto      1
olbD1OIXZQ8      1
4VkNuaGbMiY      1
mQTAJ8CrSN0      1
svxMxqG17wU      1
YAceVl9toWM      1
apREz_5r7qw      1
Qvb45V9Y6q4      1
OWgSwhMDkuY      1
KNiVrh0jZIw      1
m7W7lQjxVrU      1
vK5Z7nJmEbk      1
coHt16QE3x8      1
shndeihYRKw      1
DIRPBFLBEts      1
howQ_ZU6Qqw      1
Xq9YOaLur8s      1
UiuBuk3tGoU      1
pzo4Sb67-nI      1
cKQnWRbU9FA 